In [ ]:
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=beedbbc5677a9165c8d42ffdd277de18eee92068200c2bba5a1d2c51674a226e
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
import pyspark
print(pyspark.__version__)

3.4.0


In [ ]:
#Ajouter le jar correspondant à votre version de spark :
#(graphframes : https://spark-packages.org/package/graphframes/graphframes)
from pyspark.sql import SparkSession
spark = (SparkSession
            .builder
            .config('spark.jars.packages', 'graphframes:graphframes:0.8.2-spark3.2-s_2.12')
            .appName("tp6")
            .getOrCreate()
)

1)	Créer un DataFrame pour chaque fichier.

In [ ]:
bikeStations = spark.read.option("header","true")\
.csv("/content/station_data.csv")
tripData = spark.read.option("header","true")\
.csv("/content/trip_data.csv")

In [ ]:
tripData

DataFrame[Trip ID: string, Duration: string, Start Date: string, Start Station: string, Start Terminal: string, End Date: string, End Station: string, End Terminal: string, Bike #: string, Subscriber Type: string, Zip Code: string]

In [ ]:
from pyspark.sql.functions import col

tripData = tripData.withColumn("Duration", col("Duration").cast("int"))

In [ ]:
tripData

DataFrame[Trip ID: string, Duration: int, Start Date: string, Start Station: string, Start Terminal: string, End Date: string, End Station: string, End Terminal: string, Bike #: string, Subscriber Type: string, Zip Code: string]

2)	Dans la table des sommets (le DataFrame qui représente le fichier station_data.csv), renommer la colonne name en id.

3)	Dans la table des arcs (le DataFrame qui représente le fichier trip_data.csv), renommer la colonne Start Station en src et la colonne End Station en dst.

In [ ]:
stationVertices = bikeStations.withColumnRenamed("name", "id").distinct()
tripEdges = tripData\
.withColumnRenamed("Start Station", "src")\
.withColumnRenamed("End Station", "dst")

4)	Créer un GraphFrame, qui représente notre graphe.

In [ ]:
from graphframes import GraphFrame
stationGraph = GraphFrame(stationVertices, tripEdges)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:169: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


# Interroger le graphe

5)	Retourner le nombre de voyages effectués entre chaque source et destination triés par ordre décroissant (selon le nombre de voyages).

In [ ]:
from pyspark.sql.functions import desc
stationGraph.edges.groupBy("src", "dst")\
.count().orderBy(desc("count"))\
.show(10, False)

+---------------------------------------+----------------------------------------+-----+
|src                                    |dst                                     |count|
+---------------------------------------+----------------------------------------+-----+
|San Francisco Caltrain 2 (330 Townsend)|Townsend at 7th                         |4    |
|5th at Howard                          |San Francisco Caltrain 2 (330 Townsend) |3    |
|San Francisco Caltrain 2 (330 Townsend)|Powell Street BART                      |2    |
|2nd at Townsend                        |Market at Sansome                       |2    |
|Spear at Folsom                        |2nd at Townsend                         |2    |
|Market at Sansome                      |Broadway St at Battery St               |2    |
|Davis at Jackson                       |Embarcadero at Sansome                  |2    |
|San Francisco Caltrain 2 (330 Townsend)|2nd at South Park                       |2    |
|Steuart at Market   

6)	Retourner le nombre de voyages qui se commencent ou se terminent à partir de la station ‘Townsend at 7th’ triés par ordre décroissant (selon le nombre de voyages).

In [ ]:
stationGraph.edges\
.where("src = 'Townsend at 7th' OR dst = 'Townsend at 7th'")\
.groupBy("src", "dst").count()\
.orderBy(desc("count"))\
.show(10)

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|San Francisco Cal...|     Townsend at 7th|    4|
|       5th at Howard|     Townsend at 7th|    1|
|     Townsend at 7th|     Spear at Folsom|    1|
|     Townsend at 7th|Harry Bridges Pla...|    1|
|     Spear at Folsom|     Townsend at 7th|    1|
|       Howard at 2nd|     Townsend at 7th|    1|
+--------------------+--------------------+-----+



7) Retourner les sommets qui n'ont jamais été une destination d'un voyage qui commence à partir de 'Spear at Folsom'.

In [ ]:
# Requête avec la fonction subtract
SpearFolsomDestinations = stationGraph \
    .edges \
    .where("src = 'Spear at Folsom'") \
    .select("dst") \
    .distinct()

otherStations = stationGraph \
    .vertices \
    .select("id") \
    .distinct() \
    .subtract(SpearFolsomDestinations)

# Affichage des résultats
otherStations.show(1000, False)

+---------------------------------------------+
|id                                           |
+---------------------------------------------+
|2nd at Folsom                                |
|California Ave Caltrain Station              |
|Washington at Kearney                        |
|Powell at Post (Union Square)                |
|Golden Gate at Polk                          |
|Yerba Buena Center of the Arts (3rd @ Howard)|
|Market at Sansome                            |
|MLK Library                                  |
|Spear at Folsom                              |
|Japantown                                    |
|Commercial at Montgomery                     |
|Paseo de San Antonio                         |
|San Salvador at 1st                          |
|Rengstorff Avenue / California Street        |
|Civic Center BART (7th at Market)            |
|Ryland Park                                  |
|San Jose Diridon Caltrain Station            |
|San Jose Civic Center                  

8)	Retourner la station qui a le nombre maximum de voyages entrants.

In [ ]:
from pyspark.sql.functions import desc
stationGraph.inDegrees.orderBy(desc("inDegree")).limit(1)\
.show(1, False)

+---------------------------------------+--------+
|id                                     |inDegree|
+---------------------------------------+--------+
|San Francisco Caltrain 2 (330 Townsend)|9       |
+---------------------------------------+--------+



In [ ]:
from pyspark.sql.functions import count
from pyspark.sql.functions import desc

stationGraph.edges.groupBy("dst").agg(count('*').alias('ct')).orderBy(desc('ct')).limit(1).show()


+--------------------+---+
|                 dst| ct|
+--------------------+---+
|San Francisco Cal...|  9|
+--------------------+---+



9)	Retourner le voyage qui a la plus grande durée.

In [ ]:
stationGraph.edges.orderBy(desc("Duration")).limit(1).show()

+-------+--------+---------------+--------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|           src|Start Terminal|       End Date|                 dst|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913448|     932|8/31/2015 21:57|Post at Kearny|            47|8/31/2015 22:12|South Van Ness at...|          66|   472|     Subscriber|   94702|
+-------+--------+---------------+--------------+--------------+---------------+--------------------+------------+------+---------------+--------+



10)	Créer un sous-graphe qui ne contient que les voyages qui se commencent ou se terminent à ‘Townsend at 7th’.

In [ ]:
townAnd7thEdges = stationGraph.edges\
.where("src = 'Townsend at 7th' OR dst = 'Townsend at 7th'")

subgraph = GraphFrame(stationGraph.vertices, townAnd7thEdges)

/usr/local/lib/python3.9/dist-packages/pyspark/sql/dataframe.py:169: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


11)	Retourner tous les chemins qui forment un motif en "triangle" entre trois stations.

In [ ]:
motifs = stationGraph.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)")
motifs.show(1000, False)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+
|a                                                                                               |ab                                                                                                                                         |b                                                

12)	 Retourner tous les chemins qui passent par trois sommets et qui commencent à partir de 'Townsend at 7th'

In [ ]:
# Recherche de tous les chemins qui passent par trois sommets
paths = stationGraph.find("(a)-[]->(b); (b)-[]->(c)")

# Filtrage des chemins qui commencent à partir de 'Townsend at 7th'
filtered_paths = paths.filter("a.id = 'Townsend at 7th'")

# Affichage des résultats
filtered_paths.show(truncate=False)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+---------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|a                                                                          |b                                                                                               |c                                                                                                  |
+---------------------------------------------------------------------------+------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|{65, Townsend at 7th, 37.771058, -122.402717, 15, San Francisco, 8/22/2013}|{49, Spear at Folsom, 37.790302, -122.390637, 19, San Francisco, 8/20/2013}                     |{